In [ ]:
!unzip /content/vehicles.zip

In [1]:
import tensorflow as tf
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras import metrics

In [2]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model

In [3]:
dataset_base="./vehicles/"

In [4]:
class_number=len(os.listdir("./vehicles/train/"))

In [5]:
train_path=os.path.join(dataset_base,"train/")
val_path=os.path.join(dataset_base,"test/")

In [6]:
train_bike= os.path.join(train_path,"bike/")
train_boat = os.path.join(train_path,"boat/")
train_bus = os.path.join(train_path,"bus/")
train_car = os.path.join(train_path,"car/")
train_cycle = os.path.join(train_path,"cycle/")
train_helicopter = os.path.join(train_path,"helicopter /")
train_plane= os.path.join(train_path,"plane/")
train_scooty = os.path.join(train_path,"scooty/")
train_truck = os.path.join(train_path,"truck/")

In [7]:
print("Number of Total Class= %2d" % (class_number)) 
print("Number of Images For Class bike =%2d" % len((train_bike)))
print("Number of Images For Class boat =%2d" % len((train_boat)))
print("Number of Images For Class bus =%2d" % len((train_bus)))
print("Number of Images For Class car =%2d" % len((train_car)))
print("Number of Images For Class cycle =%2d" % len((train_cycle)))
print("Number of Images For Class helicopter =%2d" % len((train_helicopter)))
print("Number of Images For Class plane =%2d" % len((train_plane)))
print("Number of Images For Class scooty =%2d" % len((train_scooty)))
print("Number of Images For Class truck =%2d" % len((train_truck)))

Number of Total Class=  9
Number of Images For Class bike =22
Number of Images For Class boat =22
Number of Images For Class bus =21
Number of Images For Class car =21
Number of Images For Class cycle =23
Number of Images For Class helicopter =29
Number of Images For Class plane =23
Number of Images For Class scooty =24
Number of Images For Class truck =23


In [8]:
train_custom_datagen = ImageDataGenerator(
        rotation_range=30,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

In [9]:
train_custom_generator = train_custom_datagen.flow_from_directory(
    train_path,
    color_mode="rgb",
    target_size=(150, 150),
    batch_size=8,
    class_mode='categorical',
    shuffle=True,
    classes = ['bike','boat',"bus","car","cycle","helicopter","plane","scooty","truck"])

Found 468 images belonging to 9 classes.


In [10]:
val_custom_datagen = ImageDataGenerator(rescale=1./255)

In [11]:
val_custom_generator = val_custom_datagen.flow_from_directory(
    val_path,
    color_mode="rgb",
    target_size=(150, 150),
    batch_size=8,
    class_mode='categorical',
     classes = ['bike','boat',"bus","car","cycle","helicopter","plane","scooty","truck"])

Found 72 images belonging to 9 classes.


In [12]:
model=tf.keras.applications.ResNet50(
    include_top=False,
    weights="imagenet",
    input_shape=(120, 120, 3),
    pooling="avg",
    classes=9
)

In [13]:
model.trainable = False
x = tf.keras.layers.Flatten()(model.output)
x  =tf.keras.layers.Dense(512, activation='relu')(x)
x  =tf.keras.layers.Dropout(0.5)(x)
x  =tf.keras.layers.Dense(128, activation='relu')(x)
x  =tf.keras.layers.Dropout(0.5)(x)
x=tf.keras.layers.Dense(9, activation='softmax')(x)

In [14]:
model = Model(inputs=model.input, outputs=x )
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 120, 120, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 126, 126, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 60, 60, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 60, 60, 64)   256         conv1_conv[0][0]                 
______________________________________________________________________________________________

In [15]:
opt = tf.keras.optimizers.Adam(learning_rate=0.001)

In [16]:
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

In [17]:
model.fit_generator(
        train_custom_generator,
        steps_per_epoch=468 // 8,
        epochs=50,
        validation_data=val_custom_generator,
        validation_steps=72 // 8)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/50
58/58 [==============================] - 9s 74ms/step - loss: 2.4107 - accuracy: 0.1130 - val_loss: 2.2268 - val_accuracy: 0.1250
Epoch 2/50
58/58 [==============================] - 3s 54ms/step - loss: 2.2324 - accuracy: 0.1239 - val_loss: 2.1704 - val_accuracy: 0.1528
Epoch 3/50
58/58 [==============================] - 3s 55ms/step - loss: 2.2056 - accuracy: 0.1304 - val_loss: 2.1394 - val_accuracy: 0.1250
Epoch 4/50
58/58 [==============================] - 3s 53ms/step - loss: 2.1866 - accuracy: 0.1638 - val_loss: 2.2036 - val_accuracy: 0.1389
Epoch 5/50
58/58 [==============================] - 3s 53ms/step - loss: 2.1639 - accuracy: 0.1652 - val_loss: 2.0719 - val_accuracy: 0.2222
Epoch 6/50
58/58 [==============================] - 3s 53ms/step - loss: 2.0957 - accuracy: 0.1804 - val_loss: 2.0406 - val_accuracy: 0.1944
Epoch 7/50
58/58 [==============================] - 3s 54ms/step - loss: 2.0956 - accuracy: 0.1935 - val_loss: 2.0272 - val_accuracy: 0.1806
Epoch 8/50
58